In [3]:
%matplotlib inline
import pandas as pd

import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn import metrics
from sklearn.metrics import pairwise_distances
import pylab as pl
from sklearn.covariance import EllipticEnvelope
from sklearn.svm import OneClassSVM
import matplotlib.pyplot as plt
import matplotlib.font_manager

pd.set_option("display.max_rows", 16)

LARGE_FIGSIZE = (12, 8)

In [4]:
#first, read the csv file as a df
eeg_df = pd.read_csv("eeg_dataset",parse_dates = True)
eeg_df['recording_time'] = eeg_df['recording_time'].astype("datetime64[ms]")
eeg_df.columns

Index(['attention', 'delta', 'highAlpha', 'highBeta', 'lowAlpha', 'lowBeta',
       'lowGamma', 'meditation', 'midGamma', 'recording_time', 'session_id',
       'theta', 'user_activity_id', 'user_id', 'user_mental_state_id'],
      dtype='object')

In [5]:
#We will produce grpahs for each user:
#A graph will be produced for a single user for a single attribute and a constant mental state showing that particular attribute for
#the different physical states
def graph_given_users_on_the_mentioned_attributes_user_activities_mental_states(user_ids,excluded_attributes,included_user_activities,included_mental_states):
    #first, keep the attributes we are interested in
    columns = eeg_df.columns
    columns = [col for col in columns if col not in excluded_attributes]
    #print(columns)
    temp_df = eeg_df[columns]
    #Then filter out the excess user activity attributes
    temp_df = temp_df[temp_df['user_activity_id'].isin(included_user_activities)]
    #print(temp_df.head())
    #print(temp_df['user_activity_id'].unique())
    #Now keep onlythe given mental state ids
    temp_df = temp_df[temp_df['user_mental_state_id'].isin(included_mental_states)]
    
    #And of course, filter the users
    temp_df = temp_df[temp_df['user_id'].isin(user_ids)]
    #We will expand this method a lot in the future iA
    
    
    #print(temp_df['user_activity_id'].unique())
    #print(temp_df['user_mental_state_id'].unique())
    #print(temp_df['user_id'].unique())
    #print(temp_df['user_activity_id'].unique())
    #print(temp_df.columns)
    #draw_graph(0,5,['attention', 'meditation','delta', 'highAlpha', 'highBeta'],5,temp_df)
    drawValForEachAttrForAllActivities(0,5,['attention', 'meditation','delta', 'highAlpha', 'highBeta'],[5,7,8],temp_df)
    return temp_df
    
    
    
    

In [6]:
#included_attributes = [ 'attention', 'delta', 'highAlpha', 'highBeta', 'lowAlpha', 'lowBeta',
       #'lowGamma', 'meditation', 'midGamma','theta']
excluded_attributes = [ 'lowAlpha', 'lowBeta','lowGamma','midGamma']
included_user_activities = [5,7,8]
included_mental_states = [5]
user_ids = [0]
temp_df = graph_given_users_on_the_mentioned_attributes_user_activities_mental_states(user_ids,excluded_attributes,included_user_activities,included_mental_states)

NameError: name 'drawValForEachAttrForAllActivities' is not defined

In [ ]:
#Now we need to graph. 
def draw_graph(user_id,mental_state,all_attributes,all_activities,cur_df):
    cur_df = cur_df[all_attributes]
    plt.figure(); cur_df.plot();
    
        
    

In [ ]:
def drawValForEachAttrForAllActivities(user_id,mental_state,attributes_to_draw,all_activities,cur_df):
    df_map = {}
    for att in attributes_to_draw:
        df_map[att] = pd.DataFrame()
    #print(df_map)
    
    grouped_by_activity = cur_df.groupby("user_activity_id")
    
    for group_name,sub_df in grouped_by_activity:
        #print(sub_df)
        #print(group_name)
        for att in attributes_to_draw:
            col_name = att + str(group_name)
            #print(col_name)
            #print(sub_df[att])
            df_map[att][col_name] = sub_df[att].reset_index(drop=True)
    #Now replace all NAN with interpolation
    for att in attributes_to_draw:
        df_map[att] = df_map[att].interpolate()
        #plt.figure(); df_map[att].plot(logy = True);
        #break
        print(att,df_map[att].describe())
    
        
    #print(df_map)

In [7]:
def q_25(df_column):
    return df_column.quantile(0.25)
def q_50(df_column):
    return df_column.quantile(0.5)
def q_75(df_column):
    return df_column.quantile(0.75)
def prepare_dataset(cur_df):
        func = {'attention':['mean','std','max','min',q_25,q_50,q_75], 'meditation':['mean','std','max','min',q_25,q_50,q_75],'delta':['mean','std','max','min',q_25,q_50,q_75], 'highAlpha':['mean','std','max','min',q_25,q_50,q_75], 'highBeta':['mean','std','max','min',q_25,q_50,q_75], 'lowAlpha':['mean','std','max','min',q_25,q_50,q_75], 'lowBeta':['mean','std','max','min',q_25,q_50,q_75],
        'lowGamma':['mean','std','max','min',q_25,q_50,q_75],'midGamma':['mean','std','max','min',q_25,q_50,q_75],'theta':['mean','std','max','min',q_25,q_50,q_75] }
        #func = {'attention':['mean','std',q_25,q_50,q_75]}
        att_med = cur_df.groupby("user_id").agg(func)
        
        return att_med,att_med.values
        
        
            #print(group_name,sub_df)

    

In [8]:
refined_df,dataset = prepare_dataset(eeg_df)
#refined_df.head()
dataset.shape

(45, 70)

In [ ]:
eeg_df['attention'].describe()

In [9]:
def run_clustering(num_clusters,dataset):
    best_score = -1000
    best_model = None


    for i in range(100):
    
        kmeans_model = KMeans(n_clusters=num_clusters, random_state=1).fit(dataset)
   
        score = metrics.silhouette_score(dataset, kmeans_model.labels_, metric='euclidean')
    
        if(score > best_score):
            best_score = score
            best_model = kmeans_model
    print(best_score)      


    labels = best_model.labels_
    #print(labels)
    print(best_model.cluster_centers_)
    #plt.scatter(best_model.cluster_centers_[:,0],best_model.cluster_centers_[:,1])
    #plt.xlabel("CV")
    #plt.ylabel("Dailty average sales")
    #plt.scatter(np_array_of_average_and_var[:,0], np_array_of_average_and_var[:,1], c=labels)
    #plt.show()

In [13]:
#run_clustering(4,dataset)

In [1]:
def anomally_detection(dataset):
    classifiers = {
 
    
    
    "OCSVM": OneClassSVM(nu=0.261, gamma=0.05)
        }
    #"Robust covariance": EllipticEnvelope(contamination=outliers_fraction)}
    # Learn a frontier for outlier detection with several classifiers
    #xx1, yy1 = np.meshgrid(np.linspace(-8, 28, 500), np.linspace(3, 40, 500))
    #x2, yy2 = np.meshgrid(np.linspace(3, 10, 500), np.linspace(-5, 45, 500))
    
    for i, (clf_name, clf) in enumerate(classifiers.items()):
    
        clf.fit(dataset)
        #Z1 = clf.decision_function(np.c_[xx1.ravel(), yy1.ravel()])
        #Z1 = Z1.reshape(xx1.shape)
        return clf
   

In [ ]:
anomally_clf = anomally_detection(dataset)
anomally_clf.predict(dataset)